In [1]:
import pandas as pd
import numpy as np

In [2]:
data_preprocessed = pd.read_csv('C:/Users/lode/Downloads/DATASET/data_preprocessed.csv')
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


In [3]:
data_preprocessed['Absenteeism Time in Hours'].nunique()

19

In [4]:
data_preprocessed['Absenteeism Time in Hours'].unique()

array([  4,   0,   2,   8,  40,   1,   7,   3,  32,   5,  16,  24,  64,
        56,  80, 120, 112, 104,  48], dtype=int64)

In [5]:
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

In [6]:
targets = np.where(data_preprocessed['Absenteeism Time in Hours'] >
                                                         data_preprocessed['Absenteeism Time in Hours'].median(), 1, 0)

In [7]:
targets.sum() / data_preprocessed.shape[0]

0.45571428571428574

In [8]:
data_preprocessed['Excessive Absenteeism'] = targets
data_preprocessed

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,5,2,179,22,40,237.656,22,1,2,0,8,1
696,1,0,0,0,5,2,225,26,28,237.656,24,0,1,2,3,0
697,1,0,0,0,5,3,330,16,28,237.656,25,1,0,0,8,1
698,0,0,0,1,5,3,235,16,32,237.656,25,1,0,0,2,0


In [9]:
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours',
                                           'Daily Work Load Average','Distance to Work'], axis = 1)
data_with_targets.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the week,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Excessive Absenteeism
0,0,0,0,1,7,1,289,33,30,0,2,1,1
1,0,0,0,0,7,1,118,50,31,0,1,0,0
2,0,0,0,1,7,2,179,38,31,0,0,0,0
3,1,0,0,0,7,3,279,39,24,0,2,0,1
4,0,0,0,1,7,3,289,33,30,0,2,1,0


In [10]:
unscaled_inputs =  data_with_targets.iloc[:, :-1]

In [11]:
from sklearn.preprocessing import StandardScaler

absenteeism_scaler = StandardScaler()

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    def __init__(self, columns, copy=True, with_mean=True, with_std=True):
        self.columns = columns
        self.copy = copy
        self.with_mean = with_mean
        self.with_std = with_std

    def fit(self, X, y=None):
        self.scaler = StandardScaler(copy=self.copy, with_mean=self.with_mean, with_std=self.with_std)
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self

    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [13]:
unscaled_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Day of the week', 'Transportation Expense', 'Age',
       'Body Mass Index', 'Education', 'Children', 'Pets'], dtype=object)

In [14]:
columns_to_omits = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Education']

In [15]:
columns_to_scaled = [x for x in unscaled_inputs.columns.values if x not in columns_to_omits ]

In [16]:
columns_to_scaled

['Month Value',
 'Day of the week',
 'Transportation Expense',
 'Age',
 'Body Mass Index',
 'Children',
 'Pets']

In [17]:
absenteeism_scaler = CustomScaler(columns_to_scaled)

In [18]:
absenteeism_scaler.fit(unscaled_inputs)

CustomScaler(columns=['Month Value', 'Day of the week',
                      'Transportation Expense', 'Age', 'Body Mass Index',
                      'Children', 'Pets'])

In [19]:
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

In [20]:
scaled_inputs

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the week,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.182726,-0.683704,1.005844,-0.536062,0.767431,0,0.880469,0.268487
1,0,0,0,0,0.182726,-0.683704,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690
2,0,0,0,1,0.182726,-0.007725,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690
3,1,0,0,0,0.182726,0.668253,0.854936,0.405184,-0.643782,0,0.880469,-0.589690
4,0,0,0,1,0.182726,0.668253,1.005844,-0.536062,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,-0.388293,-0.007725,-0.654143,0.562059,-1.114186,1,0.880469,-0.589690
696,1,0,0,0,-0.388293,-0.007725,0.040034,-1.320435,-0.643782,0,-0.019280,1.126663
697,1,0,0,0,-0.388293,0.668253,1.624567,-1.320435,-0.408580,1,-0.919030,-0.589690
698,0,0,0,1,-0.388293,0.668253,0.190942,-0.692937,-0.408580,1,-0.919030,-0.589690


In [21]:
from sklearn.model_selection import train_test_split

In [22]:
train_test_split(scaled_inputs, targets)

[     Reason_1  Reason_2  Reason_3  Reason_4  Month Value  Day of the week  \
 211         1         0         0         0    -0.388293        -0.683704   
 70          0         0         0         1     1.039256        -0.007725   
 49          1         0         0         0     0.753746        -1.359682   
 514         0         0         0         1     1.039256         1.344231   
 389         0         0         0         1    -1.244823         1.344231   
 ..        ...       ...       ...       ...          ...              ...   
 397         0         0         0         1    -0.959313        -0.683704   
 345         1         0         0         0     1.610276         0.668253   
 136         0         0         0         1    -1.530333         0.668253   
 79          0         0         0         1     1.039256         1.344231   
 571         1         0         0         0    -1.530333         0.668253   
 
      Transportation Expense       Age  Body Mass Index  Educa

In [23]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, test_size = 0.2, random_state = 20)

In [24]:
print(x_train.shape, y_train.shape)

(560, 12) (560,)


In [25]:
print(x_test.shape, y_test.shape)

(140, 12) (140,)


In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [27]:
reg = LogisticRegression()

In [28]:
reg.fit(x_train, y_train)

LogisticRegression()

In [29]:
reg.score(x_train, y_train)

0.7732142857142857

In [30]:
model_outputs = reg.predict(x_train)
model_outputs

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,

In [31]:
y_train

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,

In [32]:
model_outputs == y_train

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True, False, False,  True,  True,  True,  True,
       False,  True, False,  True, False, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False, False, False,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True, False,  True, False,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False,  True, False,  True,  True, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,

In [33]:
np.sum(model_outputs == y_train)

433

In [34]:
model_outputs.shape[0]

560

In [35]:
np.sum(model_outputs == y_train) / model_outputs.shape[0]

0.7732142857142857

In [36]:
reg.intercept_

array([-1.65534037])

In [37]:
reg.coef_

array([[ 2.79965101,  0.93248633,  3.09464524,  0.85414844,  0.16675155,
        -0.08519984,  0.61098463, -0.16451036,  0.27091802, -0.20035475,
         0.36205145, -0.28579071]])

In [38]:
feature_names = unscaled_inputs.columns.values

In [39]:
summary_table = pd.DataFrame(columns = ['Feature Name'], data = feature_names)

summary_table['Coefficient'] = np.transpose(reg.coef_)

summary_table

,Feature Name,Coefficient
0,Reason_1,2.799651
1,Reason_2,0.932486
2,Reason_3,3.094645
3,Reason_4,0.854148
4,Month Value,0.166752
5,Day of the week,-0.085200
6,Transportation Expense,0.610985
7,Age,-0.164510
8,Body Mass Index,0.270918
9,Education,-0.200355


In [40]:
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature Name,Coefficient
0,Intercept,-1.655340
1,Reason_1,2.799651
2,Reason_2,0.932486
3,Reason_3,3.094645
4,Reason_4,0.854148
5,Month Value,0.166752
6,Day of the week,-0.085200
7,Transportation Expense,0.610985
8,Age,-0.164510
9,Body Mass Index,0.270918


In [42]:
summary_table['Odds_ratio'] = np.exp(summary_table['Coefficient'])

In [43]:
summary_table

,Feature Name,Coefficient,Odds_ratio
0,Intercept,-1.655340,0.191027
1,Reason_1,2.799651,16.438909
2,Reason_2,0.932486,2.540819
3,Reason_3,3.094645,22.079404
4,Reason_4,0.854148,2.349373
5,Month Value,0.166752,1.181461
6,Day of the week,-0.085200,0.918329
7,Transportation Expense,0.610985,1.842244
8,Age,-0.164510,0.848309
9,Body Mass Index,0.270918,1.311168


In [44]:
summary_table.sort_values(by = 'Odds_ratio', ascending = False)

,Feature Name,Coefficient,Odds_ratio
3,Reason_3,3.094645,22.079404
1,Reason_1,2.799651,16.438909
2,Reason_2,0.932486,2.540819
4,Reason_4,0.854148,2.349373
7,Transportation Expense,0.610985,1.842244
11,Children,0.362051,1.436273
9,Body Mass Index,0.270918,1.311168
5,Month Value,0.166752,1.181461
6,Day of the week,-0.085200,0.918329
8,Age,-0.164510,0.848309


In [45]:
reg.score(x_test, y_test)

0.7428571428571429

In [46]:
predicted_proba = reg.predict_proba(x_test)

In [50]:
predicted_proba

array([[0.73651067, 0.26348933],
       [0.60918875, 0.39081125],
       [0.4105826 , 0.5894174 ],
       [0.80357418, 0.19642582],
       [0.07247751, 0.92752249],
       [0.31710876, 0.68289124],
       [0.31331677, 0.68668323],
       [0.13372578, 0.86627422],
       [0.79818833, 0.20181167],
       [0.75085197, 0.24914803],
       [0.48109718, 0.51890282],
       [0.1983499 , 0.8016501 ],
       [0.07791389, 0.92208611],
       [0.70756141, 0.29243859],
       [0.30777815, 0.69222185],
       [0.57285817, 0.42714183],
       [0.5405107 , 0.4594893 ],
       [0.57140094, 0.42859906],
       [0.38274628, 0.61725372],
       [0.04850517, 0.95149483],
       [0.69874403, 0.30125597],
       [0.79432434, 0.20567566],
       [0.39647469, 0.60352531],
       [0.42433826, 0.57566174],
       [0.26730977, 0.73269023],
       [0.7571217 , 0.2428783 ],
       [0.51476324, 0.48523676],
       [0.86780288, 0.13219712],
       [0.20014864, 0.79985136],
       [0.7847556 , 0.2152444 ],
       [0.

In [48]:
predicted_proba[:, 1]

array([0.26348933, 0.39081125, 0.5894174 , 0.19642582, 0.92752249,
       0.68289124, 0.68668323, 0.86627422, 0.20181167, 0.24914803,
       0.51890282, 0.8016501 , 0.92208611, 0.29243859, 0.69222185,
       0.42714183, 0.4594893 , 0.42859906, 0.61725372, 0.95149483,
       0.30125597, 0.20567566, 0.60352531, 0.57566174, 0.73269023,
       0.2428783 , 0.48523676, 0.13219712, 0.79985136, 0.2152444 ,
       0.3730501 , 0.68587358, 0.69098238, 0.54261282, 0.20567566,
       0.50734998, 0.20958969, 0.74276806, 0.43612588, 0.59227189,
       0.22687861, 0.4354993 , 0.21589086, 0.39575367, 0.81547789,
       0.56873774, 0.69062368, 0.27481719, 0.20129583, 0.18182924,
       0.59373789, 0.34487901, 0.67029014, 0.28500273, 0.84966449,
       0.47045642, 0.88973941, 0.25447451, 0.31851838, 0.31635515,
       0.721278  , 0.65968446, 0.31019296, 0.78649971, 0.19799226,
       0.26738263, 0.08195034, 0.22930955, 0.72660557, 0.33333653,
       0.20958969, 0.29292276, 0.90863459, 0.4380138 , 0.61959

In [51]:
import pickle

In [52]:
with open('model', 'wb') as file:
    pickle.dump(reg, file)

In [53]:
with open('scaler', 'wb') as file:
    pickle.dump(absenteeism_scaler, file)